# [Optimizing the model parameters](https://learn.microsoft.com/en-us/training/modules/intro-machine-learning-pytorch/6-optimization)

In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 배치 사이즈 만큼 데이터를 가져와서 학습

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork()

## 1.Setting hyperparameters

- Number of epochs
> The number times the entire training dataset is pass through the network.
- Batch Size
> The number of data samples seen by the model in each eposh. Iterates are the number of batches needs to compete an epoch.
- Learning Rate
> The size of steps the model match as it searchs for best weights that will produce a higher model accuracy.    
> - Smaller values means the model will take a longer time to find the best weights    
> - Larger values may result in the model step over and misses the best weights 

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 2.Add an optimization loop

- The Train Loop
> iterate over the training dataset and try to converge to optimal parameters.   
- The Validation/test Loop
> iterate over the test dataset to check if model performance is improving.

### Add a loss function
- `nn.MSELoss` (Mean Square Error) used for regression tasks
- `nn.NLLLoss` (Negative Log Likelihood) used for classification
- `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`

In [7]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Backpropagation
- optimizer.zero_grad()
> 역전파 단계를 실행하기 전에 변화도(gradient)를 0으로 재설정
- loss.backward()
> 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도 계산
- optimizer.step()
> 변화도를 계산한 뒤에 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

## 3.Full implementation

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad(): #test 데이터는 학습이 되면 안되기 때문에 막는다
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310343  [    0/60000]
loss: 2.302825  [ 6400/60000]
loss: 2.297660  [12800/60000]
loss: 2.284460  [19200/60000]
loss: 2.277862  [25600/60000]
loss: 2.282296  [32000/60000]
loss: 2.253685  [38400/60000]
loss: 2.258143  [44800/60000]
loss: 2.254160  [51200/60000]
loss: 2.230529  [57600/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 0.035125 

Epoch 2
-------------------------------
loss: 2.248445  [    0/60000]
loss: 2.249213  [ 6400/60000]
loss: 2.240846  [12800/60000]
loss: 2.219539  [19200/60000]
loss: 2.205677  [25600/60000]
loss: 2.232142  [32000/60000]
loss: 2.159340  [38400/60000]
loss: 2.180644  [44800/60000]
loss: 2.182073  [51200/60000]
loss: 2.130055  [57600/60000]
Test Error: 
 Accuracy: 46.1%, Avg loss: 0.033697 

Epoch 3
-------------------------------
loss: 2.177477  [    0/60000]
loss: 2.176374  [ 6400/60000]
loss: 2.165153  [12800/60000]
loss: 2.120746  [19200/60000]
loss: 2.096515  [25600/60000]
loss: 2.165327  [32000/600

## 4.Saving Models

In [12]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 5. Loading Models

In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

### Predictions

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [15]:
import os
os.getcwd()

'c:\\tutorial\\8. 딥러닝\\7-0. Introduction_Pytorch'